In [3]:
import sys
import asyncio
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(Path("packages/demo-notebook/.env"))

REPO_ROOT = Path("../..")
BASE_URL  = "http://localhost:3000"

sys.path.insert(0, str((REPO_ROOT / "packages/demo-recorder/src").resolve()))
from demo_recorder.task_runner import run_tasks, build_prompt

print(f"Repo: {REPO_ROOT.resolve()}")
print(f"App:  {BASE_URL}")

Repo: /Users/macbookpro/Documents/projects/aura
App:  http://localhost:3000


In [2]:
# Hand this tasks array in from any upstream agent (e.g. Claude Code).
# Each description is a self-contained, browser-verifiable spec.
# All tasks merge into ONE prompt → ONE browser session → ONE video.
tasks = [
    {
        "id": "login-success",
        "description": (
            "Navigate to http://localhost:3000/login. "

            "Type 'admin' into the field with id=username and "
            "'password123' into the field with id=password. "
            "Click the button with id=sign-in-button. "
            "Confirm the element with id=success-banner is visible."
        ),
    },
    {
        "id": "login-failure",
        "description": (
            "Navigate to http://localhost:3000/login. "
            "Type 'hacker' into the field with id=username and "
            "'wrongpass' into the field with id=password. "
            "Click the button with id=sign-in-button. "
            "Confirm the element with id=error-banner is visible."
        ),
    },
]

print(build_prompt(tasks, BASE_URL))

Complete the following verification steps in order:

  Step 1 [login-success]: Navigate to http://localhost:3000/login. Type 'admin' into the field with id=username and 'password123' into the field with id=password. Click the button with id=sign-in-button. Confirm the element with id=success-banner is visible.
  Step 2 [login-failure]: Navigate to http://localhost:3000/login. Type 'hacker' into the field with id=username and 'wrongpass' into the field with id=password. Click the button with id=sign-in-button. Confirm the element with id=error-banner is visible.

Recording instructions (important):
- Complete ALL steps in sequence — do not skip any.
- Wait 1-2 seconds between each action so interactions are clearly visible.
- After filling a field, pause briefly before clicking submit.
- After each step completes, pause 2 seconds before starting the next step.
- Do not close the browser when done.


In [3]:
result = await run_tasks(tasks, base_url=BASE_URL)

verdict_icon = "✅" if result.verdict == "pass" else "❌"
print(f"\n{verdict_icon} Verdict: {result.verdict.upper() if result.verdict else 'unknown'}")
print(f"   {result.verdict_reasoning}")
print(f"\nOutput:  {result.output_path}")
for f in sorted(result.output_path.iterdir()):
    print(f"  {f.name}")

INFO     [service] Using anonymized telemetry, see https://docs.browser-use.com/development/monitoring/telemetry.
INFO     [Agent] 🎯 Task: Complete the following verification steps in order:

  Step 1 [login-success]: Navigate to http://localhost:3000/login. Type 'admin' into the field with id=username and 'password123' into the field with id=password. Click the button with id=sign-in-button. Confirm the element with id=success-banner is visible.
  Step 2 [login-failure]: Navigate to http://localhost:3000/login. Type 'hacker' into the field with id=username and 'wrongpass' into the field with id=password. Click the button with id=sign-in-button. Confirm the element with id=error-banner is visible.

Recording instructions (important):
- Complete ALL steps in sequence — do not skip any.
- Wait 1-2 seconds between each action so interactions are clearly visible.
- After filling a field, pause briefly before clicking submit.
- After each step completes, pause 2 seconds before starting the 

Running 2 task(s) in one session → /Users/macbookpro/Documents/projects/aura/demos/2026-02-28-234700

Prompt preview:
------------------------------------------------------------
Complete the following verification steps in order:

  Step 1 [login-success]: Navigate to http://localhost:3000/login. Type 'admin' into the field with id=username and 'password123' into the field with id=password. Click the button with id=sign-in-button. Confirm the element with id=success-banner is visible.
  Step 2 [login-failure]: Navigate to http://localhost:3000/login. Type 'hacker' into the field with id=username and 'wrongpass' into the field with id=password. Click the button with id=sign-in-button. Confirm the element with id=error-banner is visible.

Recording instructions (important):
- Complete ALL steps in sequence — do not skip any.
- Wait 1-2 seconds between each action so interactions are clearly visible.
- After filling a field, pause briefly before clicking submit.
- After each step complet

INFO     [Agent] Starting a browser-use agent with version 0.12.0, with provider=browser-use and model=bu-1-0
INFO     [BrowserSession] 📹 Started/Switched video recording to target F81FE4EAE1845E62E9AD6BF021C2933E
INFO     [Agent]   ▶️   navigate: url: http://localhost:3000, new_tab: False
WARNING  [BrowserSession] ⚠️ Page readiness timeout (8.0s, 8111ms) for http://localhost:3000
INFO     [tools] 🔗 Navigated to http://localhost:3000
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   🧠 Memory: The page is still loading or empty at the root URL. I need to proceed with Step 1, which requires navigating to http://localhost:3000/login. I will create a todo list to track the verification steps as requested.
INFO     [Agent]   ▶️  [1/2] write_file: file_name: todo.md, content: - [ ] Step 1: Successful login verification\n- [ ] Step 2: Failed login verification
INFO     [tools] 💾 Data written to file todo.md successfully. File location: /var/folders/8g/fvcm01m93jxg0m39qv8nsk7800

✅ Judge verdict: PASS — The agent successfully completed both verification steps in the correct order. In Step 1, it navigated to the login page
✓ Recording complete → /Users/macbookpro/Documents/projects/aura/demos/2026-02-28-234700

✅ Verdict: PASS
   The agent successfully completed both verification steps in the correct order. In Step 1, it navigated to the login page, entered the correct credentials ('admin'/'password123'), and clicked sign-in. Screenshot 3 clearly shows the 'Authentication successful!' banner (id=success-banner). In Step 2, it navigated back to the login page, entered incorrect credentials ('hacker'/'wrongpass'), and clicked sign-in. Screenshot 5 shows the 'Invalid username or password' banner (id=error-banner). The agent also followed the recording instructions by waiting between actions (2-second pauses between typing and clicking) and between steps. The final output confirms the visibility of both elements as requested.

Output:  /Users/macbookpro/Documents/pr

In [ ]:
# ─── Skills: record once, replay deterministically ───────────────────────────
# Skills are a browser-use CLOUD feature (browser_use_sdk, not browser_use).
# They record a UI flow once and turn it into a fast, deterministic endpoint.
#
# Use when:
#   - You have a stable flow you want to run on every PR cheaply
#   - You want to detect DOM regressions (skill fails = something broke)
#   - Flow doesn't need LLM reasoning on each run
#
# Don't use when:
#   - The UI is actively being redesigned (re-record after stabilising)
#   - You need adaptive reasoning (use run_tasks instead)

from browser_use_sdk import AsyncBrowserUse

client = AsyncBrowserUse()  # reads BROWSER_USE_API_KEY from env

# ── Step 1: Create the skill (record once — run this once per flow) ──────────
# NOTE: requires a publicly reachable URL (not localhost).
# For local dev, use ngrok: `ngrok http 3000` → get https://abc.ngrok.io
SKILL_URL = BASE_URL  # swap with your ngrok / Vercel preview URL

skill = await client.skills.create(
    goal=(
        "Test the login flow at this URL. "
        "Accept two parameters: username (str) and password (str). "
        "Fill the #username field, fill the #password field, click #sign-in-button. "
        "Return: {passed: bool, banner_text: str} based on what banner appears."
    ),
    agent_prompt=(
        f"Go to {SKILL_URL}/login. "
        "Fill the username field with 'admin', the password field with 'password123'. "
        "Click the Sign In button. Wait for the result banner to appear."
    ),
)

print(f"Skill created: {skill.id}")
print("Save this ID — use it to replay the flow on every PR without re-recording.")


In [ ]:
# ── Step 2: Replay the skill (fast, deterministic, no LLM reasoning) ─────────
# This is what runs on every PR. Swap skill.id with the saved ID string.

# Test: valid credentials
result_pass = await client.skills.execute(
    skill.id,
    parameters={"username": "admin", "password": "password123"},
)
print(f"✅ Valid login:   {result_pass}")

# Test: invalid credentials
result_fail = await client.skills.execute(
    skill.id,
    parameters={"username": "wrong", "password": "bad"},
)
print(f"❌ Invalid login: {result_fail}")

# If either call throws / returns unexpected output → DOM changed → regression caught.
# Re-record the skill with client.skills.create() after intentional UI changes.


In [4]:
# ─── Daytona: Create Snapshot from Dockerfile ────────────────────────────────
# Run this cell ONCE to create/update the demo-recorder snapshot in Daytona.
# After this, you can spin up sandboxes with the "demo-recorder" snapshot.
#
# Prerequisites:
#   - DAYTONA_API_KEY set in .env (or as environment variable)
#   - pip install daytona

import os
from daytona import Daytona, Image, CreateSnapshotParams

# Initialize Daytona client (reads DAYTONA_API_KEY from env automatically)
daytona_client = Daytona()

# Read the Dockerfile content
DOCKERFILE_PATH = REPO_ROOT / "packages/demo-recorder/Dockerfile.daytona"

# Build image from Dockerfile
image = Image.from_dockerfile(str(DOCKERFILE_PATH))

# Create (or update) the snapshot
try:
    snapshot = daytona_client.snapshot.create(
        CreateSnapshotParams(name="demo-recorder", image=image),
        on_logs=lambda chunk: print(chunk, end=""),
    )
    print(f"\n✅ Snapshot created: {snapshot.name}")
except Exception as e:
    print(f"❌ Failed to create snapshot: {e}")

56c42440: Already exists er (SnapshotState.PENDING)Creating snapshot demo-recorder (SnapshotState.BUILDING)Step 1/8 : FROM python:3.12-slim3.12-slim: Pulling from library/python
a546a81ba946:Already exists
90b93dccd534:Already exists
2d517a4d89f5:Already existsDigest: sha256:f3fa41d74a768c2fce8016b98c191ae8c1bacd8f1152870a3f9f87d350920b7cStatus: Downloaded newer image for python:3.12-slim ---> 5265ced2e36cStep 2/8 : RUN apt-get update && apt-get install -y     git curl ffmpeg     libnss3 libatk1.0-0 libatk-bridge2.0-0 libcups2 libdrm2     libxkbcommon0 libxcomposite1 libxdamage1 libxfixes3     libxrandr2 libgbm1 libasound2 libpango-1.0-0 libcairo2     && rm -rf /var/lib/apt/lists/* ---> Running in 49506c4c9873Hit:1 http://deb.debian.org/debian trixie InRelease
Get:2 http://deb.debian.org/debian trixie-updates InRelease [47.3 kB]Get:3 http://deb.debian.org/debian-security trixie-security InRelease [43.4 kB]Get:4 http://deb.debian.org/debian trixie/main amd64 Packages [9670 kB]Get:5 http

In [ ]:
# ─── Daytona: Run demo-recorder in a Sandbox ─────────────────────────────────
# Spin up an isolated sandbox and run Claude Code + demo-recorder inside it.

import json

# Create sandbox from the snapshot
sandbox = daytona_client.create()  # Uses default snapshot, or pass params

print(f"✅ Sandbox created: {sandbox.id}")

# Run demo-recorder via Claude Code
tasks_json = json.dumps(tasks)
cmd = f'python -m demo_recorder.cli --base-url {BASE_URL} --tasks \'{tasks_json}\''

result = sandbox.process.code_run(cmd)

if result.exit_code != 0:
    print(f"❌ Error: {result.exit_code}")
print(result.result)

# Cleanup (optional - comment out to keep sandbox for debugging)
# daytona_client.delete(sandbox)
# print("Sandbox deleted")